In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [10]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [11]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [12]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110610

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,52929.0324
2018-02-28,36568.0692
2018-03-31,48765.5787
2018-04-30,56104.3011
2018-05-31,51848.3063
2018-06-30,47259.3437
2018-07-31,52554.9802
2018-08-31,57281.1522
2018-09-30,42601.0196


In [13]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    52929.0324
 2018-02-28    36568.0692
 2018-03-31    48765.5787
 2018-04-30    56104.3011
 2018-05-31    51848.3063
 2018-06-30    47259.3437
 2018-07-31    52554.9802
 2018-08-31    57281.1522
 2018-09-30    42601.0196
 2018-10-31    43637.9304
 2018-11-30    45395.0415
 2018-12-31    44475.7805
 2019-01-31    58062.2408
 2019-02-28    41265.4058
 2019-03-31    45725.5423
 2019-04-30    48037.5212
 2019-05-31    54534.3718
 2019-06-30    39525.8832
 2019-07-31    48748.4472
 2019-08-31    54112.8722
 2019-09-30    36227.0917
 2019-10-31    52358.5408
 2019-11-30    26873.5951
 2019-12-31    35167.3622
 2020-01-31    37069.1757
 2020-02-29    48762.3463
 2020-03-31    45258.0789
 2020-04-30    39109.4597
 2020-05-31    35597.4954
 2020-06-30    28156.8752
 2020-07-31    49277.5746
 2020-08-31    52461.1032
 2020-09-30    42330.2628
 2020-10-31    39573.4998
 2020-11-30    37090.5412
 2020-12-31    33901.8797
 2021-01-31    34544.3893
 2021-02-

# ARIMA

In [14]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [15]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -5.316799
p-value : 0.000005
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [16]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,1,0)(0,0,0)[1] intercept   : AIC=875.196, Time=0.07 sec
 ARIMA(0,1,1)(0,0,0)[1] intercept   : AIC=864.868, Time=0.04 sec
 ARIMA(0,1,2)(0,0,0)[1] intercept   : AIC=866.371, Time=0.30 sec
 ARIMA(0,1,3)(0,0,0)[1] intercept   : AIC=873.820, Time=0.07 sec
 ARIMA(0,1,4)(0,0,0)[1] intercept   : AIC=877.290, Time=0.10 sec
 ARIMA(0,1,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.50 sec
 ARIMA(1,1,0)(0,0,0)[1] intercept   : AIC=865.562, Time=0.04 sec
 ARIMA(1,1,1)(0,0,0)[1] intercept   : AIC=865.867, Time=0.26 sec
 ARIMA(1,1,2)(0,0,0)[1] intercept   : AIC=inf, Time=0.41 sec
 ARIMA(1,1,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.55 sec
 ARIMA(1,1,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.51 sec
 ARIMA(2,1,0)(0,0,0)[1] intercept   : AIC=866.534, Time=0.05 sec
 ARIMA(2,1,1)(0,0,0)[1] intercept   : AIC=868.756, Time=0.13 sec
 ARIMA(2,1,2)(0,0,0)[1] intercept   : AIC=inf, Time=0.49 sec
 ARIMA(2,1,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.60 sec
 ARIMA(3,1,0)(0,0,0)[1] intercept   : AIC=868.969

ARIMA(order=(0, 1, 1), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [17]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(0,1,1))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                   41
Model:                 ARIMA(0, 1, 1)   Log Likelihood                -423.267
Method:                       css-mle   S.D. of innovations           7037.836
Date:                Sat, 11 Sep 2021   AIC                            852.533
Time:                        04:09:30   BIC                            857.674
Sample:                             1   HQIC                           854.405
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       -276.7740     89.594     -3.089      0.002    -452.375    -101.173
ma.L1.D.y     -1.0000      0.072    -13.986      0.000      -1.140      -0.860
                                    Roots           

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [18]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([37993.79650066, 37717.02251096]), array([7037.83635837, 7037.83635837]), array([[24199.89070918, 51787.70229214],
       [23923.11671948, 51510.92830245]]))
